In [1]:
import socket
import uuid

EXP_ID = str(uuid.uuid4())
PROJECT = "siim-isic-melanoma-classification"
NB = "exp0000"
DESCRIPTION = "test notebook"
HOST = socket.gethostname()
NB, HOST, EXP_ID

('exp0000', 'ccb91580f89c', '51100d20-c24f-4c69-b39a-8179a4dcb846')

In [ ]:
# from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
import gc
import glob
import io
import os
import pickle
import random
import shutil
import warnings
from collections import OrderedDict
from pathlib import Path

import albumentations as albu
import cv2
import h5py  # HDF5のライブラリ
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import timm
import torch
import torch.optim as optim
from PIL import Image
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score
from sklearn.model_selection import KFold, StratifiedGroupKFold, StratifiedKFold
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

warnings.simplefilter("ignore")


ROOT_DIR = Path("../")
DATA_DIR = ROOT_DIR / "data"
OUTPUT_DIR = ROOT_DIR / "output"
CP_DIR = OUTPUT_DIR / "checkpoint"


def to_pickle(filename, obj):
    with open(filename, mode="wb") as f:
        pickle.dump(obj, f)


def unpickle(filename):
    with open(filename, mode="rb") as fo:
        p = pickle.load(fo)
    return p

In [ ]:
ROOT_DIR = Path("../")
DATA_DIR = ROOT_DIR / Path("data")

In [ ]:
import multiprocessing

print("cpu count:", multiprocessing.cpu_count())


class Config:
    N_LABEL = 10
    N_FOLD = 5
    RANDOM_SATE = 42
    LR = 1.0e-05
    MAX_LR = 8.0e-5
    PATIENCE = 6
    EPOCH = 10
    BATCH_SIZE = 288
    SKIP_EVALUATE_NUM = 0
    BACK_BONE = "tf_efficientnet_b1_ns"
    RUN_FOLD_COUNT = 10
    IMG_SIZE = 139
    T_MAX = 20
    ETA_MIN = 3.0e-7
    SCHEDULER_GAMMA = 1.0
    ACCUMULATION_STEMP = 2
    NUM_WORKERS = multiprocessing.cpu_count()


def seed_everything(seed=1234):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(seed=Config.RANDOM_SATE)

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)


seed_everything(Config.RANDOM_SATE)

In [ ]:
train_df = pl.read_csv(DATA_DIR / "train-metadata.csv")

In [ ]:
train_df.select([pl.col(name).n_unique().alias(name) for name in train_df.columns])

In [ ]:
train_df = pl.read_csv(DATA_DIR / "train-metadata.csv")
test_df = pl.read_csv(DATA_DIR / "test-metadata.csv")

display(train_df.shape)
display(train_df.head(3))
display(test_df.shape)
display(test_df.head(3))

In [ ]:
train_df.filter(pl.col("target") == 1).shape

In [ ]:
train_df = train_df.with_columns(
    pl.col("age_approx").cast(pl.String).replace("NA", np.nan).cast(pl.Float64),
)

test_df = test_df.with_columns(
    pl.col("age_approx").cast(pl.String).replace("NA", np.nan).cast(pl.Float64),
)

In [ ]:
err = 1e-5

num_cols = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+ 
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
]

new_num_cols = [
    'lesion_size_ratio',             # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'lesion_shape_index',            # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
    'hue_contrast',                  # tbp_lv_H                - tbp_lv_Hext              abs
    'luminance_contrast',            # tbp_lv_L                - tbp_lv_Lext              abs
    'lesion_color_difference',       # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt  
    'border_complexity',             # tbp_lv_norm_border      + tbp_lv_symm_2axis
    'color_uniformity',              # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max

    'position_distance_3d',          # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
    'perimeter_to_area_ratio',       # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'area_to_perimeter_ratio',       # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'lesion_visibility_score',       # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'symmetry_border_consistency',   # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'consistency_symmetry_border',   # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)

    'color_consistency',             # tbp_lv_stdL             / tbp_lv_Lext
    'consistency_color',             # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    'size_age_interaction',          # clin_size_long_diam_mm  * age_approx
    'hue_color_std_interaction',     # tbp_lv_H                * tbp_lv_color_std_mean
    'lesion_severity_index',         # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
    'shape_complexity_index',        # border_complexity       + lesion_shape_index
    'color_contrast_index',          # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm

    'log_lesion_area',               # tbp_lv_areaMM2          + 1  np.log
    'normalized_lesion_size',        # clin_size_long_diam_mm  / age_approx
    'mean_hue_difference',           # tbp_lv_H                + tbp_lv_Hext    / 2
    'std_dev_contrast',              # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
    'color_shape_composite_index',   # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
    'lesion_orientation_3d',         # tbp_lv_y                , tbp_lv_x  np.arctan2
    'overall_color_difference',      # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3

    'symmetry_perimeter_interaction',# tbp_lv_symm_2axis       * tbp_lv_perimeterMM
    'comprehensive_lesion_index',    # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
    'color_variance_ratio',          # tbp_lv_color_std_mean   / tbp_lv_stdLExt
    'border_color_interaction',      # tbp_lv_norm_border      * tbp_lv_norm_color
    'border_color_interaction_2',
    'size_color_contrast_ratio',     # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
    'age_normalized_nevi_confidence',# tbp_lv_nevi_confidence  / age_approx
    'age_normalized_nevi_confidence_2',
    'color_asymmetry_index',         # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max

    'volume_approximation_3d',       # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'color_range',                   # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
    'shape_color_consistency',       # tbp_lv_eccentricity     * tbp_lv_color_std_mean
    'border_length_ratio',           # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
    'age_size_symmetry_index',       # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
    'index_age_size_symmetry',       # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
]

In [ ]:
train_df = train_df.with_columns(
        pl.col('age_approx').cast(pl.String).replace('NA', np.nan).cast(pl.Float64),
    ).with_columns(
        pl.col(pl.Float64).fill_nan(pl.col(pl.Float64).median()), # You may want to impute test data with train
    ).with_columns(
        lesion_size_ratio              = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
        lesion_shape_index             = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
        hue_contrast                   = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
        luminance_contrast             = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
        lesion_color_difference        = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
        border_complexity              = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
        color_uniformity               = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
    ).with_columns(
        position_distance_3d           = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
        perimeter_to_area_ratio        = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
        area_to_perimeter_ratio        = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
        lesion_visibility_score        = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
        combined_anatomical_site       = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
        symmetry_border_consistency    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
        consistency_symmetry_border    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border')),
    ).with_columns(
        color_consistency              = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
        consistency_color              = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
        size_age_interaction           = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
        hue_color_std_interaction      = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
        lesion_severity_index          = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
        shape_complexity_index         = pl.col('border_complexity') + pl.col('lesion_shape_index'),
        color_contrast_index           = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
    ).with_columns(
        log_lesion_area                = (pl.col('tbp_lv_areaMM2') + 1).log(),
        normalized_lesion_size         = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
        mean_hue_difference            = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
        std_dev_contrast               = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
        color_shape_composite_index    = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
        lesion_orientation_3d          = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
        overall_color_difference       = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
    ).with_columns(
        symmetry_perimeter_interaction = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
        comprehensive_lesion_index     = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
        color_variance_ratio           = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
        border_color_interaction       = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
        border_color_interaction_2     = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
        size_color_contrast_ratio      = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
        age_normalized_nevi_confidence = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
        age_normalized_nevi_confidence_2 = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
        color_asymmetry_index          = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
    ).with_columns(
        volume_approximation_3d        = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
        color_range                    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
        shape_color_consistency        = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
        border_length_ratio            = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
        age_size_symmetry_index        = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
        index_age_size_symmetry        = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
    ).with_columns(
            ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
    ).with_columns(
        count_per_patient = pl.col('isic_id').count().over('patient_id'),
    )

In [ ]:
test_df = test_df.with_columns(
        pl.col('age_approx').cast(pl.String).replace('NA', np.nan).cast(pl.Float64),
    ).with_columns(
        pl.col(pl.Float64).fill_nan(pl.col(pl.Float64).median()), # You may want to impute test data with train
    ).with_columns(
        lesion_size_ratio              = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
        lesion_shape_index             = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
        hue_contrast                   = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
        luminance_contrast             = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
        lesion_color_difference        = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
        border_complexity              = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
        color_uniformity               = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
    ).with_columns(
        position_distance_3d           = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
        perimeter_to_area_ratio        = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
        area_to_perimeter_ratio        = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
        lesion_visibility_score        = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
        combined_anatomical_site       = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
        symmetry_border_consistency    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
        consistency_symmetry_border    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border')),
    ).with_columns(
        color_consistency              = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
        consistency_color              = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
        size_age_interaction           = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
        hue_color_std_interaction      = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
        lesion_severity_index          = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
        shape_complexity_index         = pl.col('border_complexity') + pl.col('lesion_shape_index'),
        color_contrast_index           = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
    ).with_columns(
        log_lesion_area                = (pl.col('tbp_lv_areaMM2') + 1).log(),
        normalized_lesion_size         = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
        mean_hue_difference            = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
        std_dev_contrast               = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
        color_shape_composite_index    = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
        lesion_orientation_3d          = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
        overall_color_difference       = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
    ).with_columns(
        symmetry_perimeter_interaction = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
        comprehensive_lesion_index     = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
        color_variance_ratio           = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
        border_color_interaction       = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
        border_color_interaction_2     = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
        size_color_contrast_ratio      = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
        age_normalized_nevi_confidence = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
        age_normalized_nevi_confidence_2 = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
        color_asymmetry_index          = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
    ).with_columns(
        volume_approximation_3d        = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
        color_range                    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
        shape_color_consistency        = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
        border_length_ratio            = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
        age_size_symmetry_index        = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
        index_age_size_symmetry        = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
    ).with_columns(
            ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
    ).with_columns(
        count_per_patient = pl.col('isic_id').count().over('patient_id'),
    )

In [ ]:
USE_COL1 = ['age_approx_patient_norm',  'tbp_lv_H_patient_norm', 'tbp_lv_H', 'clin_size_long_diam_mm', 'count_per_patient', 'tbp_lv_y_patient_norm', 'normalized_lesion_size', 'position_distance_3d_patient_norm', 'color_uniformity_patient_norm', 'tbp_lv_Hext', 'lesion_visibility_score_patient_norm', 'age_normalized_nevi_confidence_2']
USE_COL2 = ['tbp_lv_H', 'clin_size_long_diam_mm', 'tbp_lv_perimeterMM', 'tbp_lv_minorAxisMM', 'tbp_lv_areaMM2']
USE_COL = list(set(USE_COL1) | set(USE_COL2))

In [ ]:
train_df.select(pl.col(USE_COL).std())

In [ ]:
train_df.select(pl.col(USE_COL).mean())

In [ ]:
#, 'mean_hue_difference_patient_norm', 'mean_hue_difference'
train_df[USE_COL].to_pandas().corr().style.background_gradient(cmap='viridis')

In [ ]:
from sklearn.preprocessing import StandardScaler
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
scaler = StandardScaler()
scaler.fit(train_df.select(USE_COL))

train_df = train_df.with_columns(
    pl.DataFrame(scaler.transform(train_df.select(USE_COL)), schema=USE_COL)
)

test_df = test_df.with_columns(
    pl.DataFrame(scaler.transform(test_df.select(USE_COL)), schema=USE_COL)
)

In [ ]:
train_df = train_df.with_columns(
    pl.col(USE_COL).fill_nan(0)
)

test_df = test_df.with_columns(
    pl.col(USE_COL).fill_nan(0)
)

In [ ]:
train_df.select(pl.col(USE_COL).mean())

In [ ]:
def pil2cv(image):
    """PIL型 -> OpenCV型"""
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image


def read_images_from_hdf5(file_path):
    with h5py.File(file_path, "r") as file:
        ids_list = list(file.keys())
        ids_images = {}
        for img_id in tqdm(ids_list):
            image_data = file[img_id][()]
            image = pil2cv(Image.open(io.BytesIO(image_data)))
            ids_images[img_id] = np.array(image)

    return ids_images

In [ ]:
for i in test_df.iter_rows():
    print(i)

In [ ]:
%%time
'''img_map = {}
img_map = read_images_from_hdf5(DATA_DIR / "train-image.hdf5")

test_img_map = read_images_from_hdf5(DATA_DIR / "test-image.hdf5")
img_map.update(test_img_map)
'''
img_map = unpickle("../data/img_map.pkl")

In [ ]:
list(img_map.values())[0].shape

In [ ]:
def get_augmentation():
    train_transform = [
        albu.Transpose(p=0.5),
        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),
        #albu.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.3, p=0.5),
        albu.RandomBrightnessContrast (brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),
        # albu.LongestMaxSize(max_size=Config.IMG_SIZE, always_apply=False),
        albu.RandomResizedCrop(
            p=0.5, scale=[0.8, 1.0], height=Config.IMG_SIZE, width=Config.IMG_SIZE
        ),
        albu.ShiftScaleRotate(
            p=0.5,
            shift_limit=0.2,
            scale_limit=0.2,
            rotate_limit=90,
            border_mode=0,
            value=0,
            mask_value=0,
        ),
        # albu.Cutout(num_holes=3, max_h_size=20, max_w_size=20, fill_value=0, p=0.5),
        albu.Resize(height=Config.IMG_SIZE, width=Config.IMG_SIZE),
        # albu.PadIfNeeded(always_apply=True, min_height=Config.IMG_SIZE, min_width=Config.IMG_SIZE, border_mode=2),
        #albu.Cutout(max_h_size=int(Config.IMG_SIZE * 0.375), max_w_size=int(Config.IMG_SIZE * 0.375), num_holes=1, p=0.7), 
        albu.CoarseDropout(max_height=int(Config.IMG_SIZE * 0.175), max_width=int(Config.IMG_SIZE * 0.175), max_holes=1, p=0.4),
        albu.Normalize(),
        # albu.ToSepia(p=0.3),
        # albu.ToGray(p=1)
    ]
    return albu.Compose(train_transform)


def get_test_augmentation():
    train_transform = [
        # albu.HorizontalFlip (p=0.5),
        # albu.VerticalFlip(p=0.3),
        # albu.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, p=0.5),
        # albu.LongestMaxSize(max_size=Config.IMG_SIZE, always_apply=False),
        albu.Resize(height=Config.IMG_SIZE, width=Config.IMG_SIZE),
        # albu.PadIfNeeded(always_apply=True, min_height=Config.IMG_SIZE, min_width=Config.IMG_SIZE, border_mode=2),
        # albu.HorizontalFlip(p=0.4),
        albu.Normalize(),
        # albu.VerticalFlip(p=0.3),
        # albu.ShiftScaleRotate(p=0.3, shift_limit=0.2, scale_limit=0.2, rotate_limit=1, border_mode=0, value=0, mask_value=0),
        # albu.RandomResizedCrop(p=0.5, scale=[0.9, 1.0], height=Config.IMG_SIZE, width=Config.IMG_SIZE),
        # albu.ToSepia(p=0.3),
        # albu.ToGray(p=1)
    ]
    return albu.Compose(train_transform)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import torch


class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, adaptive=False, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, adaptive=adaptive, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups
        self.defaults.update(self.base_optimizer.defaults)

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None:
                    continue
                self.state[p]["old_p"] = p.data.clone()
                e_w = (
                    (torch.pow(p, 2) if group["adaptive"] else 1.0)
                    * p.grad
                    * scale.to(p)
                )
                p.add_(e_w)  # climb to the local maximum "w + e(w)"

        if zero_grad:
            self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue
                p.data = self.state[p]["old_p"]  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad:
            self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert (
            closure is not None
        ), "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(
            closure
        )  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][
            0
        ].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
            torch.stack(
                [
                    ((torch.abs(p) if group["adaptive"] else 1.0) * p.grad)
                    .norm(p=2)
                    .to(shared_device)
                    for group in self.param_groups
                    for p in group["params"]
                    if p.grad is not None
                ]
            ),
            p=2,
        )
        return norm

    def load_state_dict(self, state_dict):
        super().load_state_dict(state_dict)
        self.base_optimizer.param_groups = self.param_groups

In [ ]:
def evaluation(true, pred):
    y_hat = pred
    min_tpr = 0.80
    max_fpr = abs(1 - min_tpr)

    v_gt = abs(true - 1)
    v_pred = np.array([1.0 - x for x in y_hat])
    # v_pred = 1.0 - y_hat

    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (
        partial_auc_scaled - 0.5
    )

    return partial_auc

from torchmetrics.classification import BinaryAUROC

def evaluation_torch(true, pred):
    min_tpr = 0.80
    max_fpr = 1 - min_tpr

    # AUROCメトリクスの初期化
    auroc = BinaryAUROC(max_fpr=max_fpr)

    # メトリクスをデバイスに移動
    auroc = auroc.to(device)

    # 1 - trueと1 - predを使用してAUROCを計算
    v_gt = torch.abs(true - 1)
    v_pred = 1.0 - pred

    partial_auc = auroc(v_pred, v_gt.int())

    # スケーリングされた部分的なAUCを計算
    partial_auc_scaled = (partial_auc - 0.5 * max_fpr) / (1 - max_fpr)

    # 最終的な部分的なAUCを計算
    final_partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (
        partial_auc_scaled - 0.5
    )

    return partial_auc

In [ ]:
import pytorch_lightning as L
from pytorch_lightning import LightningDataModule, LightningModule
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, TQDMProgressBar, LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger

class CVDataSet(Dataset):
    COLUMN_NAME = USE_COL
    
    def __init__(self, df, images, transforms, data_type=None):
        self.df = df
        self.images = images
        self.transforms = transforms
        self.data_type = data_type
        self.target = df["target"].to_numpy()
        self.features = df.select(self.COLUMN_NAME).to_numpy()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # target
        if self.data_type in ["train", "valid"]:
            target = self.target[idx]
        else:
            target = -1

        # image
        img = img_map[self.df["isic_id"][idx]]

        augmented = self.transforms(image=img)
        img = augmented["image"]
        img = np.moveaxis(img, 2, 0)

        # tabular
        x = self.features[idx]
        
        return img, x, target

class CVNet(nn.Module):
    def __init__(self, num_features=len(CVDataSet.COLUMN_NAME)):
        super(CVNet, self).__init__()

        self.base_model = timm.create_model(
            Config.BACK_BONE, num_classes=0, pretrained=True, in_chans=3
        )
        base_model_features = self.base_model.num_features

        num_features_all = int(num_features * 3) + base_model_features

        self.img_layer = nn.Sequential(
            nn.Linear(num_features, int(num_features * 2)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features * 2), int(num_features * 3)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
        )
        
        self.cls = nn.Sequential(
            nn.Linear(num_features_all, int(num_features_all / 2)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features_all / 2), int(num_features_all / 4)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features_all / 4), int(num_features_all / 8)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features_all / 8), 1),
        )

    def forward(self, x, img):
        img_out = self.base_model(img)

        x = self.img_layer(x)

        x = torch.cat([img_out, x], 1)
        x = self.cls(x)

        return x

# define the LightningModule
class CVModule(LightningModule):
    def __init__(self, model):
        super(CVModule, self).__init__()
        self.model = model
        self.validation_step_outputs = []
        self.criterion = nn.BCEWithLogitsLoss()

        self.save_hyperparameters()
        self.automatic_optimization = False # SAM Optimizerを使うので

    def forward(self, x, img):
        return self.model(x, img)
    
    def training_step(self, batch, batch_idx):
        target, pred, loss_1, pauc = self._get_preds_loss_metric(batch)

        # SAM Optimizerを使うのでbackward, optimizer.step, optimizer.zero_grad, scheduler.stepを手動でやる。
        optimizer = self.optimizers()
        self.manual_backward(loss_1)
        optimizer.first_step(zero_grad=True)

        target, pred, loss_2, pauc = self._get_preds_loss_metric(batch)
        self.manual_backward(loss_2)

        optimizer.second_step(zero_grad=True)
        optimizer.zero_grad()
        
        self.lr_schedulers().step()
        
        self.log("train_step_loss_first_step", loss_1)
        self.log("train_step_loss", loss_2)
        self.log("train_step_metric", pauc)
        return loss_1

    def configure_optimizers(self):
        base_optimizer = optim.AdamW
        optimizer = SAM(
            self.model.parameters(), base_optimizer, lr=Config.LR, weight_decay=1.0e-02
        )
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            total_steps=self.trainer.estimated_stepping_batches,
            max_lr=Config.MAX_LR,
            pct_start=0.1,
            anneal_strategy="cos",
            div_factor=1.0e3,
            final_div_factor=1.0e3,
        )
        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def validation_step(self, batch, batch_index):            
        target, pred, loss, pauc = self._get_preds_loss_metric(batch)
        pred = pred.sigmoid()
        self.log("val_step_loss", loss)
        self.log("val_step_metric", pauc)
        self.validation_step_outputs.append(
            {"val_loss": loss, "pred": pred, "y": target}
        )

    def on_validation_epoch_start(self) -> None:
        self.validation_step_outputs.clear()  # free memory

    def on_validation_epoch_end(self) -> None:
        if trainer.global_step == 0:
            wandb.define_metric("val_metric", summary="max")
            
        outputs = self.validation_step_outputs
        pred = torch.hstack([x["pred"] for x in outputs])
        y = torch.hstack([x["y"] for x in outputs])
        pred_np = pred.to("cpu").detach().numpy().copy()
        y = y.to("cpu").detach().numpy().copy()
        pauc = evaluation(y, pred_np)
        pauc_torch = torch.tensor([pauc])

        print(
            f"val_metric - {self.current_epoch}: metric:{pauc:.6f}"
        )
        self.log("val_metric", pauc_torch)

    # metricも計算すべき
    def _get_preds_loss_metric(self, batch):
        """train/valid/test ステップが似ているための便利な関数"""
        img, x, target = batch
        img, x, target = img.float(),  x.float(), target.float()
        pred = self.model(x, img).squeeze()
        loss = self.criterion(pred, target)
        metrics_value = evaluation_torch(target, pred)
        return target, pred, loss, metrics_value

# データローダーの設定
class DataModule(LightningDataModule):
    def __init__(self, train_loader, valid_loader):
        super().__init__()
        self.train_loader = train_loader
        self.valid_loader = valid_loader

    def train_dataloader(self):
        return self.train_loader

    def val_dataloader(self):
        return self.valid_loader

In [ ]:
import wandb

skf = StratifiedGroupKFold(
    n_splits=Config.N_FOLD, random_state=Config.RANDOM_SATE, shuffle=True
)

for fold, (train_index, test_index) in enumerate(
    skf.split(train_df, train_df["target"], groups=train_df["patient_id"])
):
    print(f"====== {fold} ======")

    train, valid = train_df[train_index], train_df[test_index]

    train_dataset = CVDataSet(train, img_map, get_augmentation(), data_type="train")
    valid_dataset = CVDataSet(
        valid, img_map, get_test_augmentation(), data_type="valid"
    )

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=Config.BATCH_SIZE,
        pin_memory=True,
        shuffle=True,
        drop_last=True,
        num_workers=Config.NUM_WORKERS,
    )
    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size=Config.BATCH_SIZE,
        pin_memory=True,
        num_workers=Config.NUM_WORKERS,
    )

    data_module = DataModule(train_loader=train_dataloader, valid_loader=valid_dataloader)
    model = CVModule(CVNet())

    # モデルの訓練
    checkpoint_callback = ModelCheckpoint(
        dirpath="../output/checkpoint", # wandbLogger使っていると機能しない？？
        # filename=f'{NB}-{fold}-{{epoch}}-{{val_metric:.2f}}',
        filename=f"{NB}-{fold}",
        monitor = "val_metric",
        save_top_k=1,
        mode="max",
        save_last=False,
        enable_version_counter=False # ファイル名が重複する場合v0とかをつけない。上書く。
    )

    earystopping = EarlyStopping(
        monitor="val_metric",
        patience=Config.PATIENCE,
        mode="max",
    )

    lr_monitor = LearningRateMonitor()
    
    wandb_logger = WandbLogger(project=PROJECT, group=f"{NB}_{EXP_ID}", name=f"fold-{fold}", save_dir="../output")

    # モデルの訓練
    trainer = L.Trainer(
        accelerator="gpu",
        max_epochs=Config.EPOCH,
        check_val_every_n_epoch=1,
        num_sanity_val_steps=0,
        log_every_n_steps=1,
        callbacks=[lr_monitor, earystopping, checkpoint_callback],
        logger=wandb_logger,
    )
    trainer.fit(model, datamodule=data_module)

    wandb.finish()
    
    del (
        model,
        data_module,
        trainer,
        valid_dataloader,
        train_dataloader,
        train_dataset,
        valid_dataset,
    )
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
%%time
oof = np.zeros(len(train_df))
cv_scores = {}
skf = StratifiedGroupKFold(
    n_splits=Config.N_FOLD, random_state=Config.RANDOM_SATE, shuffle=True
)

for fold, (train_index, test_index) in enumerate(
    skf.split(train_df, train_df["target"], groups=train_df["patient_id"])
):
    print(f"====== {fold} ======")

    valid = train_df[test_index]
    valid_target = valid["target"].to_numpy()

    # TODO DataLoaderはDataModuleに色々まかせたい
    valid_dataset = CVDataSet(
        valid, img_map, get_test_augmentation(), data_type="valid"
    )
    validloader = DataLoader(
        valid_dataset,
        batch_size=Config.BATCH_SIZE,
        pin_memory=True,
        num_workers=Config.NUM_WORKERS,
    )

    module = CVModule(CVNet())
    module.load_state_dict(
        torch.load(f"../output/checkpoint/{NB}-{fold}.ckpt")["state_dict"]
    )
    module.to(device).eval()

    preds = []
    n_iter_val = len(validloader)
    for i, (img, x, target) in tqdm(enumerate(validloader), total=n_iter_val):
        with torch.no_grad():
            img, x, target = (
                img.to(device).float(),
                x.to(device).float(),
                target.to(device).float(),
            )
            outputs = module(x, img).squeeze().sigmoid()
            outputs_np = outputs.to("cpu").detach().numpy().copy()
            preds.append(outputs_np)

    pauc = evaluation(valid_target, np.hstack(preds))
    print(pauc)

    oof[test_index] = np.hstack(preds).reshape(-1)
    cv_scores[f"cv{fold}"] = pauc

    del module, valid_dataset, validloader
    torch.cuda.empty_cache()
    gc.collect()